# Import

In [9]:
import torch
import numpy as np
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import torch.nn as nn
import torch.nn.functional as F

import os
os.environ["TORCHDYNAMO_DISABLE"] = "1" 

# DNN ARCHITECTURE

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, num_convs):
        super().__init__()
        self.convs = nn.Sequential()
        
        # Membuat layer konvolusi sesuai jumlah yang ditentukan
        for i in range(num_convs):
            input_channels = in_channels if i == 0 else out_channels
            self.convs.add_module(f'conv{i+1}', nn.Conv2d(
                input_channels, out_channels, kernel_size=3, padding=1))
            self.convs.add_module(f'bn{i+1}', nn.BatchNorm2d(out_channels))
            self.convs.add_module(f'relu{i+1}', nn.ReLU(inplace=True))
        
        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Skip connection dengan 1x1 conv untuk menyesuaikan dimensi
        self.skip = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        identity = self.skip(x)  # Proses skip connection
        out = self.convs(x)      # Jalur utama melalui konvolusi
        out = self.pool(out)     # Pooling setelah konvolusi
        out += identity          # Tambahkan skip connection
        return F.relu(out)       # Aktivasi akhir

class ResidualVGG16(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        
        # Membangun blok-blok residual sesuai arsitektur VGG16
        self.blocks = nn.Sequential(
            ResidualBlock(3, 32, 2),       # Blok 1: 2 konvolusi 64 channel
            ResidualBlock(32, 64, 2),     # Blok 2: 2 konvolusi 128 channel
            ResidualBlock(64, 128, 3),    # Blok 3: 3 konvolusi 256 channel
            ResidualBlock(128, 256, 3),    # Blok 4: 3 konvolusi 512 channel
            ResidualBlock(256, 256, 3),    # Blok 5: 3 konvolusi 512 channel
        )
        
        # Classifier dengan fully-connected layers
        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.blocks(x)               # Loloskan melalui semua blok residual
        x = torch.flatten(x, 1)          # Flatten feature maps
        x = self.classifier(x)           # Loloskan melalui classifier
        return x



In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Inisialisasi model (harus sama dengan arsitektur saat training)
model = ResidualVGG16(num_classes=2).to(device)

# Load weights
model.load_state_dict(torch.load('best_model.pth'))
model.eval()  # Set model ke mode evaluasi

C:\Users\ASUS\AppData\Local\Temp\ipykernel_17004\2347091581.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


ResidualVGG16(
  (blocks): Sequential(
    (0): ResidualBlock(
      (convs): Sequential(
        (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
      )
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (skip): Sequential(
        (0): Conv2d(3, 32, kernel_size=(1, 1), stride=(2, 2))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ResidualBlock(
      (convs): Sequential(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, a

# Load Data Test

In [7]:
# 2. Persiapan Data Test
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

test_dataset = datasets.ImageFolder(
    root='Dataset/final/test',  # Ganti dengan path dataset test
    transform=test_transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False  # Jangan di-shuffle untuk evaluasi
)

# Testing Model

In [10]:
# 3. Prediksi pada Data Test
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 4. Tampilkan Classification Report
print(classification_report(
    all_labels,
    all_preds,
    target_names=test_dataset.classes  # Ambil nama kelas dari folder
))

# (Opsional) Hitung Accuracy
accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
print(f'Overall Accuracy: {accuracy:.4f}')

              precision    recall  f1-score   support

          no       0.98      0.97      0.98       249
         yes       0.97      0.98      0.98       249

    accuracy                           0.98       498
   macro avg       0.98      0.98      0.98       498
weighted avg       0.98      0.98      0.98       498

Overall Accuracy: 0.9759
